<a href="https://colab.research.google.com/github/Nov05/Google-Colaboratory/blob/master/20231214_chatgpt_3_5_turbo_%2B_llama_index_(RAG)_with_the_book_of_mormon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **\<TOP\>**  
* 2023-12-14 modified by nov05   
* go to [the short course folder](https://drive.google.com/drive/folders/1NUa9wsPDILnlhnSZu95iylnccYYzv27g)  
* go to [the github backup](https://github.com/Nov05/Google-Colaboratory/blob/master/20231214_chatgpt_3_5_turbo_%2B_llama_index_(RAG)_with_the_book_of_mormon.ipynb)  
* 20231214_L3-Sentence_window_retrieval.ipynb  

---

* [check openai billing](https://platform.openai.com/account/billing/overview) (might need to turn off the vpn web protection)
* [Using the Book of Mormon to Answer Life’s Questions](https://www.churchofjesuschrist.org/youth/activities/new/using-the-book-of-mormon-to-answer-lifes-questions?lang=eng)    
* [Imagine with Meta AI](https://imagine.meta.com/?prompt=LDS+Mormons+in+the+year+2450)  
* Youtube: [LLama2 + RAG with BAAI embeddings](https://www.youtube.com/watch?v=WXGfV-hMtCA), by DLExplorers  
* [Enhancing Semantic Search with BAAI Embedding Model and Ontology Enrichment Using Vector Database (Chroma DB)](https://medium.com/@amanahluwalia/enhancing-semantic-search-with-baai-embedding-model-and-ontology-enrichment-using-vector-database-b660e900292c)  
* https://huggingface.co/BAAI Beijing Academy of Artificial Intelligence (智源研究所)  
* reddit: [I made a Cloudflare Workers AI script for baai/bge-small-en-v1.5, alternative to OpenAI embeddings API](https://www.reddit.com/r/LangChain/comments/18haz37/i_made_a_cloudflare_workers_ai_script_for/)

In [ ]:
## download a folder
# !gdown --no-check-certificate --folder https://drive.google.com/drive/folders/10LpvZD_trQ7t0J3NeMoP6zFvU8ZmwvCv
# !cp /content/l3_files/* /content
# !rm -r /content/l3_files
## download a file, utils.py
!gdown --no-check-certificate 'https://drive.google.com/uc?export=download&id=1qHEVMw6wZV7NdOV3E68nZySkmvBhPZ5-'
## questions.txt
!gdown --no-check-certificate 'https://drive.google.com/uc?export=download&id=1Fvx4Hz05if6ZXUllG2rcAY7RWnMWduVP'
## The Book of Mormon_50 pages.pdf
!gdown --no-check-certificate 'https://drive.google.com/uc?export=download&id=1akgig6-Qp7AYEFiBS5oWO8bXs2oRh--4'
## the full book, 558 pages
!wget -O 'The Book of Mormon.pdf' 'https://www.holybooks.com/wp-content/uploads/Book-of-Mormon-PDF.pdf'

In [ ]:
%%capture
!pip install openai==1.3.5
!pip install llama-index==0.9.8
!pip install trulens-eval==0.18.1
!pip install python-dotenv
!pip install pypdf
!pip uninstall transformers ## 4.35.2 pre-installed by colab
!pip install transformers==4.33.2
!pip install sentence-transformers==2.2.2
## restart the session

In [1]:
from google.colab import output
output.enable_custom_widget_manager()
# output.disable_custom_widget_manager()

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
# os.environ["OPENAI_API_KEY"]

# **Lesson 3: Sentence Window Retrieval**

In [4]:
import utils
import os
import openai
from google.colab import userdata
openai.api_key = utils.get_openai_api_key()

In [7]:
%%time
from llama_index import SimpleDirectoryReader
# path = '/content/The Book of Mormon_50 pages.pdf'
path = '/content/The Book of Mormon.pdf'
documents = SimpleDirectoryReader(
    input_files=[path]
).load_data()
print(type(documents))
print(len(documents))
print(type(documents[0]))
print(documents[0])

<class 'list'>
558
<class 'llama_index.schema.Document'>
Doc ID: 92841508-7293-489e-9d96-5be12306716f
Text: The   Book of MorMon Another T estament of   Jesus Christ
Published by   The Church of Jesus Christ of Latter-day Saints Salt
Lake City, Utah, USA


In [8]:
## merge documents into one
from llama_index import Document
document = Document(text="\n\n".join([doc.text for doc in documents]))

In [9]:
import os
from llama_index import ServiceContext, VectorStoreIndex, StorageContext
from llama_index.node_parser import SentenceWindowNodeParser
from llama_index.indices.postprocessor import MetadataReplacementPostProcessor
from llama_index.indices.postprocessor import SentenceTransformerRerank
from llama_index import load_index_from_storage


def build_sentence_window_index(
    documents,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5", ## Beijing Academy of Artificial Intelligence general embedding
    sentence_window_size=3,
    save_dir="sentence_index",
):
    ## create the sentence window node parser w/ default settings
    node_parser = SentenceWindowNodeParser.from_defaults(
        window_size=sentence_window_size,
        window_metadata_key="window",
        original_text_metadata_key="original_text",
    )
    sentence_context = ServiceContext.from_defaults(
        llm=llm,
        embed_model=embed_model,
        node_parser=node_parser,
    )
    if not os.path.exists(save_dir):
        sentence_index = VectorStoreIndex.from_documents(
            documents, service_context=sentence_context
        )
        sentence_index.storage_context.persist(persist_dir=save_dir)
    else:
        sentence_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=save_dir),
            service_context=sentence_context,
        )

    return sentence_index


def get_sentence_window_query_engine(
    sentence_index, similarity_top_k=6, rerank_top_n=2
):
    # define postprocessors
    postproc = MetadataReplacementPostProcessor(target_metadata_key="window")
    rerank = SentenceTransformerRerank(
        top_n=rerank_top_n, model="BAAI/bge-reranker-base"
    )

    sentence_window_engine = sentence_index.as_query_engine(
        similarity_top_k=similarity_top_k, node_postprocessors=[postproc, rerank]
    )
    return sentence_window_engine

In [10]:
# %%time
# from llama_index.llms import OpenAI
# index = build_sentence_window_index(
#     [document],
#     llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),
#     save_dir="./sentence_index",
# )

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

CPU times: user 42min 3s, sys: 1min 2s, total: 43min 5s
Wall time: 43min 55s


In [ ]:
# %%time
# query_engine = get_sentence_window_query_engine(index, similarity_top_k=6)

# **TruLens Evaluation**  

In [11]:
%%time
eval_questions = []
path = '/content/questions.txt'
with open(path, 'r') as file:
    for line in file:
        ## Remove newline character and convert to integer
        item = line.strip()
        eval_questions.append(item)

from trulens_eval import Tru
def run_evals(eval_questions, tru_recorder, query_engine):
    for question in eval_questions:
        with tru_recorder as recording:
            response = query_engine.query(question)

from utils import get_prebuilt_trulens_recorder
from trulens_eval import Tru
Tru().reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.
CPU times: user 70.5 ms, sys: 6.26 ms, total: 76.8 ms
Wall time: 170 ms


# **Sentence window size = 3**  

In [13]:
%%time
sentence_index_3 = build_sentence_window_index(
    [document], # documents,
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),
    embed_model="local:BAAI/bge-small-en-v1.5",
    sentence_window_size=3,
    save_dir="sentence_index_3",
)
sentence_window_engine_3 = get_sentence_window_query_engine(
    sentence_index_3
)
tru_recorder_3 = get_prebuilt_trulens_recorder(
    sentence_window_engine_3,
    app_id='sentence window engine 3'
)
## OpenAIError: The api_key client option must be set either by passing api_key to the client
##              or by setting the OPENAI_API_KEY environment variable
## to process the full book:
##     CPU times: user 41min 44s, sys: 21.3 s, total: 42min 5s
##     Wall time: 42min 43s

config.json:   0%|          | 0.00/799 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
CPU times: user 41min 44s, sys: 21.3 s, total: 42min 5s
Wall time: 42min 43s


In [16]:
%%time
run_evals(eval_questions, tru_recorder_3, sentence_window_engine_3)

CPU times: user 2min 3s, sys: 661 ms, total: 2min 4s
Wall time: 2min 29s


In [15]:
Tru().run_dashboard()
## the full book: Total Tokens 5.49k
## Answer Relevance 0.99 ✅ high
## Context Relevance 0.04 🛑 low
## Groundedness 0.71 ⚠️ medium

Starting dashboard ...
npx: installed 22 in 5.693s

Go to this url and submit the ip given here. your url is: https://public-results-wash.loca.lt

  Submit this IP Address: 34.91.164.32



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

<img src="https://github.com/Nov05/pictures/blob/master/DLAI/20231213_Building%20and%20Evaluating%20Advanced%20RAG/2023-12-14%2011_01_34-Settings.jpg?raw=true" width=800>  

<img src="https://github.com/Nov05/pictures/blob/master/DLAI/20231213_Building%20and%20Evaluating%20Advanced%20RAG/2023-12-14%2011_10_03-Settings.jpg?raw=true" width=800>  

<img src="https://github.com/Nov05/pictures/blob/master/DLAI/20231213_Building%20and%20Evaluating%20Advanced%20RAG/2023-12-14%2011_14_04-Evaluations.jpg?raw=true" width=800>

In [17]:
## display output texts
import textwrap
records, feedback = Tru().get_records_and_feedback(app_ids=[]) ## "records" is a pandas dataframe
n_rows = 10
for i in range(n_rows):
    input, output = records.loc[i, ['input', 'output']]
    print('👉', textwrap.fill(input, 100))
    print('▪️', textwrap.fill(output, 100))
    output = records.loc[i+n_rows, 'output']
    print('▪️', textwrap.fill(output, 100), '\n')
## generated with the full book RAG

👉 "What is the purpose of life? (Alma 34)"
▪️ "The purpose of life, according to Alma 34, is to believe in Christ."
▪️ "The purpose of life, according to Alma 34, is to believe in Christ and have faith in His words. By
doing so, individuals can have their burdens made light and experience joy through the Son of God." 

👉 "Does God know me? (Alma 5:38, 58)"
▪️ "Yes, God knows you."
▪️ "Yes, God knows you." 

👉 "How does God answer prayers? (Enos 1)"
▪️ "God answers prayers by speaking to individuals in their minds. In the case of Enos, he prayed
mightily and received a remission of his sins. The voice of the Lord came into his mind, promising
salvation for the Lamanites in the future."
▪️ "God answers prayers by speaking to individuals in their minds. In the case of Enos, he prayed
mightily and received a remission of his sins. The voice of the Lord came into his mind, promising
salvation for the Lamanites in the future. Enos rejoiced in his Redeemer." 

👉 "How can a belief in Jesus Chr

🟢 **Texts generated from 50 pages RAG**    
```
👉 "What is the purpose of life? (Alma 34)"
▪️ "The purpose of life is not mentioned in the given context information."
▪️ "The purpose of life is not directly mentioned in the given context information."

👉 "Does God know me? (Alma 5:38, 58)"
▪️ "Yes, God knows you. (Alma 5:38, 58)"
▪️ "Yes, God knows you. (Alma 5:38, 58)"

👉 "How does God answer prayers? (Enos 1)"
▪️ "God answers prayers by communicating with individuals through visions and messengers. In Enos 1,
the individual had a vision where a messenger related to him the commandments and instructions from
God. The individual then obeyed and went to his father to share the vision, and his father confirmed
that it was from God. This suggests that God answers prayers by providing guidance and direction
through divine communication."
▪️ "God answers prayers by communicating with individuals through visions and messengers. In Enos 1,
the individual had a vision and received commandments from a messenger sent by God. This shows that
God can answer prayers by providing guidance and instructions through supernatural means."

👉 "How can a belief in Jesus Christ help me? (Alma 36)"
▪️ "Believing in Jesus Christ can bring happiness and joy into one's life. It can also help in
developing a closer relationship with God and experiencing His love. Additionally, a belief in Jesus
Christ can provide guidance and direction in making choices and decisions."
▪️ "Believing in Jesus Christ can bring happiness and joy into one's life. It can also help strengthen
family relationships and foster love within the family. Additionally, a belief in Jesus Christ can
provide a sense of purpose and direction, as well as a closer relationship with God. Ultimately,
having faith in Jesus Christ can lead to salvation and eternal life."

👉 "What does Jesus Christ expect of me? (2 Nephi 9)"
▪️ "Jesus Christ expects you to diligently seek Him and His teachings. He desires that you have faith
in Him and strive to keep His commandments. He wants you to come to know Him through the power of
the Holy Ghost, which is a gift from God. By seeking Him and following His teachings, you can
receive His tender mercies and be blessed with deliverance."
▪️ "Jesus Christ expects you to diligently seek Him and His teachings. He desires that you have faith
in Him and strive to keep His commandments. He wants you to come to know Him through the power of
the Holy Ghost, which is a gift from God. Jesus Christ expects you to repent of your sins and rely
on His mercy and grace for forgiveness. He also desires that you have a sincere desire to follow Him
and to be obedient to His teachings."

👉 "Is there life after death? (Alma 40)"
▪️ "Yes, there is a discussion about life after death in Alma 40."
▪️ "Yes, there is a discussion about life after death in Alma 40."

👉 "Why does God allow evil and suffering to occur? (2 Nephi 2; Alma 14:9\u201311; 60:13)"
▪️ "God allows evil and suffering to occur because it is a consequence of the agency or free will that
He has given to His children. People have the ability to choose between good and evil, and sometimes
they choose to do wicked things that result in suffering and evil in the world. However, God also
provides a way for individuals to overcome evil and find redemption through the Atonement of Jesus
Christ."
▪️ "God allows evil and suffering to occur as a consequence of the agency or free will that He has
given to His children. This agency allows individuals to choose between good and evil, and with that
freedom comes the potential for both positive and negative outcomes. The Book of Mormon teaches that
God does not interfere with the choices and actions of individuals, but rather allows them to
experience the consequences of their choices. This includes the suffering and evil that may result
from the wickedness and abominations of individuals. However, it is also taught that God provides a
way for individuals to overcome evil and find redemption through faith in Jesus Christ."

👉 "How can I find peace and joy? (Mosiah 2, 4)"
▪️ "By following the teachings and commandments found in the Book of Mormon, specifically in Mosiah
chapters 2 and 4, you can find peace and joy. These chapters contain teachings about serving others,
repentance, and relying on the Atonement of Jesus Christ. By applying these principles in your life,
you can experience the peace and joy that come from living a righteous and fulfilling life."
▪️ "By following the teachings and commandments found in the Book of Mormon, specifically in Mosiah
chapters 2 and 4, you can find peace and joy. These chapters contain teachings about serving others,
being humble, and turning to God for guidance and forgiveness. By applying these principles in your
life, you can experience the peace and joy that come from living a righteous and fulfilling life."

👉 "How can my family be happier and more united? (Mosiah 2)"
▪️ "By keeping the commandments and being faithful, your family can experience greater happiness and
unity. It is important to teach and remind each other of the things that the Lord has commanded.
Additionally, offering sacrifices and expressing gratitude can also contribute to a happier and more
united family."
▪️ "By keeping the commandments and being faithful, your family can experience greater happiness and
unity. It is important to teach your children to follow the Lord's commandments and to be obedient.
By doing so, you can receive blessings and guidance from the Lord, just as Nephi did when he prayed
in faith. Additionally, showing love and forgiveness towards one another can help strengthen your
family bonds."

👉 "How can I avoid sin? (Helaman 5)"
▪️ "By keeping the commandments, one can avoid sin."
▪️ "By keeping the commandments, one can avoid sin."  
```

## **single questions**

In [ ]:
%%time
window_response = sentence_window_engine_3.query(
    "What does Jesus Christ expect of me? (2 Nephi 9)"
)
display_response(window_response)

**`Final Response:`** Jesus Christ expects you to diligently seek Him and His teachings. He desires that you have faith in Him and strive to keep His commandments. He wants you to come unto Him and receive the blessings of redemption and deliverance that He offers.

CPU times: user 11.5 s, sys: 2.41 s, total: 13.9 s
Wall time: 19.9 s


In [18]:
%%time
from llama_index.response.notebook_utils import display_response
window_response = sentence_window_engine_3.query(
    "What is the purpose of life? (Alma 34)"
)
display_response(window_response)

**`Final Response:`** The purpose of life, according to Alma 34, is to believe in Christ.

CPU times: user 6.75 s, sys: 17.4 ms, total: 6.77 s
Wall time: 9.01 s


🟢 **with 50 pages RAG:**  
```
Final Response: The purpose of life is not mentioned in the given context information.
CPU times: user 8.09 s, sys: 156 ms, total: 8.24 s
Wall time: 21.2 s
```

In [ ]:
%%time
question = 'Summarize The First Book of Nephi.' ## check page 23
window_response = sentence_window_engine_3.query(question)
display_response(window_response)

**`Final Response:`** The First Book of Nephi is an account of Lehi and his family, including his wife Sariah and his four sons: Laman, Lemuel, Sam, and Nephi. The book begins with Lehi being warned by the Lord to leave Jerusalem due to his prophecies about the people's iniquity and their desire to harm him. Lehi and his family embark on a three-day journey into the wilderness. Later in the book, Nephi witnesses the power of God with the Gentiles who had been delivered from captivity. He also sees that they prosper in the land and encounters an angel who asks him if he knows the meaning of a book, to which Nephi responds that he does not.

CPU times: user 5.6 s, sys: 36.2 ms, total: 5.63 s
Wall time: 21.6 s


In [ ]:
%%time
question = 'Summarize 1 Nephi Chapter 3.' ## check page 27
window_response = sentence_window_engine_3.query(question)
display_response(window_response)

**`Final Response:`** In 1 Nephi Chapter 3, Nephi persuades his brothers to be faithful in keeping the commandments of God. They gather their gold, silver, and precious things and return to the house of Laban.

CPU times: user 7.86 s, sys: 615 ms, total: 8.48 s
Wall time: 32.8 s


In [19]:
%%time
question = 'What is the afterlife like according to Alma 40?'
window_response = sentence_window_engine_3.query(question)
display_response(window_response)

**`Final Response:`** According to Alma 40, the afterlife is described as a state where individuals are either in a state of happiness or in a state contrary to the nature of happiness. It is implied that those who are in a state contrary to the nature of happiness are in a state of bitterness and iniquity, without God in the world. The concept of restoration is also mentioned, suggesting that individuals will be rewarded or punished based on their actions in life.

CPU times: user 7.62 s, sys: 17.1 ms, total: 7.64 s
Wall time: 11.5 s


🟢 **with 50 pages RAG:**  
```
Final Response: The afterlife according to Alma 40 is not mentioned in the given context information.
CPU times: user 7.76 s, sys: 71.6 ms, total: 7.83 s
Wall time: 10.3 s
```

In [ ]:
%%time
question = 'dfafadf' ## random characters
window_response = sentence_window_engine_3.query(question)
display_response(window_response)

**`Final Response:`** I'm sorry, but I cannot provide an answer to the query "dfafadf" based on the given context information.

CPU times: user 6.95 s, sys: 26 ms, total: 6.97 s
Wall time: 11.6 s


# **Lesson 4: Auto-merging Retrieval**  

In [20]:
%%time
from llama_index.node_parser import HierarchicalNodeParser
from llama_index.node_parser import get_leaf_nodes
## create the hierarchical node parser w/ default settings
node_parser = HierarchicalNodeParser.from_defaults(
    chunk_sizes=[2048, 512, 128]
)
nodes = node_parser.get_nodes_from_documents([document])
leaf_nodes = get_leaf_nodes(nodes)
print(leaf_nodes[30].text)

The Te STimOny Of Three Wi TneSSeS
Be it known unto all nations, kindreds, tongues, and people, unto whom  
   this work shall come: That we, through the grace of God the Father, 
and our Lord Jesus Christ, have seen the plates which contain this record,  
which is a record of the people of Nephi, and also of the Lamanites, their 
brethren, and also of the people of Jared, who came from the tower of which hath been spoken.
CPU times: user 19 s, sys: 42.7 ms, total: 19.1 s
Wall time: 20 s


In [21]:
nodes_by_id = {node.node_id: node for node in nodes}
parent_node = nodes_by_id[leaf_nodes[30].parent_node.node_id]
print(parent_node.text)

In due course the plates were delivered to Joseph Smith, who translated 
them by the gift and power of God. The record is now published in many languages as a new and additional witness that Jesus Christ is the Son of the living God and that all who will come unto Him and obey the laws and ordinances of His gospel may be saved.
Concerning this record the Prophet Joseph Smith said: “I told the breth-
ren that the Book of Mormon was the most correct of any book on earth, and the keystone of our religion, and a man would get nearer to God by abiding by its precepts, than by any other book.”
In addition to Joseph Smith, the Lord provided for eleven others to see 
the gold plates for themselves and to be special witnesses of the truth and divinity of the Book of Mormon. Their written testimonies are included herewith as “The Testimony of Three Witnesses” and “The Testimony of Eight Witnesses.”
We invite all men everywhere to read the Book of Mormon, to ponder 
in their hearts the message it

In [22]:
import os

from llama_index import (
    ServiceContext,
    StorageContext,
    VectorStoreIndex,
    load_index_from_storage,
)
from llama_index.node_parser import HierarchicalNodeParser
from llama_index.node_parser import get_leaf_nodes
from llama_index import StorageContext, load_index_from_storage
from llama_index.retrievers import AutoMergingRetriever
from llama_index.indices.postprocessor import SentenceTransformerRerank
from llama_index.query_engine import RetrieverQueryEngine


def build_automerging_index(
    documents,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index",
    chunk_sizes=None,
):
    chunk_sizes = chunk_sizes or [2048, 512, 128]
    node_parser = HierarchicalNodeParser.from_defaults(chunk_sizes=chunk_sizes)
    nodes = node_parser.get_nodes_from_documents(documents)
    leaf_nodes = get_leaf_nodes(nodes)
    merging_context = ServiceContext.from_defaults(
        llm=llm,
        embed_model=embed_model,
    )
    storage_context = StorageContext.from_defaults()
    storage_context.docstore.add_documents(nodes)

    if not os.path.exists(save_dir):
        automerging_index = VectorStoreIndex(
            leaf_nodes, storage_context=storage_context, service_context=merging_context
        )
        automerging_index.storage_context.persist(persist_dir=save_dir)
    else:
        automerging_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=save_dir),
            service_context=merging_context,
        )
    return automerging_index


def get_automerging_query_engine(
    automerging_index,
    similarity_top_k=12,
    rerank_top_n=6,
):
    base_retriever = automerging_index.as_retriever(similarity_top_k=similarity_top_k)
    retriever = AutoMergingRetriever(
        base_retriever, automerging_index.storage_context, verbose=True
    )
    rerank = SentenceTransformerRerank(
        top_n=rerank_top_n, model="BAAI/bge-reranker-base"
    )
    auto_merging_engine = RetrieverQueryEngine.from_args(
        retriever, node_postprocessors=[rerank]
    )
    return auto_merging_engine

In [27]:
# %%time
# from llama_index.llms import OpenAI
# index = build_automerging_index(
#     [document],
#     llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),
#     save_dir="./merging_index",
# )
# query_engine = get_automerging_query_engine(index, similarity_top_k=6)

In [25]:
from trulens_eval import Tru
Tru().reset_database()

## **Two layers**   

In [26]:
%%time
auto_merging_index_0 = build_automerging_index(
    documents,
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index_0",
    chunk_sizes=[2048,512],
)
auto_merging_engine_0 = get_automerging_query_engine(
    auto_merging_index_0,
    similarity_top_k=12,
    rerank_top_n=6,
)
from utils import get_prebuilt_trulens_recorder
tru_recorder = get_prebuilt_trulens_recorder(
    auto_merging_engine_0,
    app_id ='auto mergine engine 0'
)
## CPU times: user 16min 51s, sys: 4min 45s, total: 21min 36s
## Wall time: 22min

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
CPU times: user 16min 51s, sys: 4min 45s, total: 21min 36s
Wall time: 22min


In [28]:
eval_questions = []
path = '/content/questions.txt'
with open(path, 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        eval_questions.append(item)

def run_evals(eval_questions, tru_recorder, query_engine):
    for question in eval_questions:
        with tru_recorder as recording:
            response = query_engine.query(question)

In [33]:
%%time
run_evals(eval_questions, tru_recorder, auto_merging_engine_0)
## CPU times: user 4min 45s, sys: 631 ms, total: 4min 46s
## Wall time: 5min 25s

> Merging 2 nodes into parent node.
> Parent node id: 4bdec105-ec2e-46c1-9ec3-62df0444fa8a.
> Parent node text: 313 ALMA  42  : 23–31
a punishment affixed, and a b repen -
tance granted; which repentance, 
mer...

> Merging 2 nodes into parent node.
> Parent node id: ac58c751-c92a-42ee-a131-10308b1e2225.
> Parent node text: 293 ALMA  34  : 1– 10
life. And then may God grant unto 
you that your d burdens may be  
light, ...

> Merging 2 nodes into parent node.
> Parent node id: 7857ee7b-cc2f-4c3a-aa1b-c6a7a2d62b9b.
> Parent node text: 115 2 NEPHI  32  : 1–9
is b none other way nor c name given 
under heaven whereby man can be 
sav...

> Merging 2 nodes into parent node.
> Parent node id: 732be7cd-268a-4d9a-ade7-1be7e0bd27eb.
> Parent node text: 239 ALMA  12  : 17–26
then is a time that whosoever dieth 
in his sins, as to a temporal b death,...

> Merging 2 nodes into parent node.
> Parent node id: 52e941db-ca10-4db6-8cbd-387a5e506f6f.
> Parent node text: 58 2 NEPHI  2  : 12–20
wherefore

In [30]:
from trulens_eval import Tru
Tru().run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   Submit this IP Address: 34.91.164.32



<Popen: returncode: 0 args: ['streamlit', 'run', '--server.headless=True', '...>

In [34]:
Tru().get_leaderboard(app_ids=[])

,Context Relevance,Groundedness,Answer Relevance,latency,total_cost
app_id,,,,,
auto mergine engine 0,0.125,0.934211,1.0,30.6,0.005005


In [35]:
## display output texts
import textwrap
records, feedback = Tru().get_records_and_feedback(app_ids=[]) ## "records" is a pandas dataframe
n_rows = 10
for i in range(n_rows):
    input, output = records.loc[i, ['input', 'output']]
    print('👉', textwrap.fill(input, 100))
    print('▪️', textwrap.fill(output, 100))
    output = records.loc[i+n_rows, 'output']
    print('▪️', textwrap.fill(output, 100), '\n')
## generated with the full book RAG

👉 "What is the purpose of life? (Alma 34)"
▪️ "The purpose of life, according to the given context, is to prepare to meet God and to work out
one's salvation with fear before Him. This is a time for individuals to have faith, repent, and
plant the word of God in their hearts. By doing so, they can experience the joy of God's Son and
have their burdens made light."
▪️ "The purpose of life, according to the given context, is to prepare to meet God and to work out
one's salvation with fear before God. This life is seen as a time for individuals to prepare
themselves through faith and repentance. The eternal plan of redemption is based on these
principles, and it is through the atonement of Jesus Christ that individuals can find joy and have
their burdens made light." 

👉 "Does God know me? (Alma 5:38, 58)"
▪️ "Yes, God knows you."
▪️ "Yes, God knows you." 

👉 "How does God answer prayers? (Enos 1)"
▪️ "God answers prayers by speaking to individuals through their minds. In the case of Enos

## **Three layers**  

In [36]:
%%time
auto_merging_index_1 = build_automerging_index(
    documents,
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index_1",
    chunk_sizes=[2048,512,128],
)
auto_merging_engine_1 = get_automerging_query_engine(
    auto_merging_index_1,
    similarity_top_k=12,
    rerank_top_n=6,
)
tru_recorder = get_prebuilt_trulens_recorder(
    auto_merging_engine_1,
    app_id ='auto merging engine 1'
)

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
CPU times: user 17min 4s, sys: 2.77 s, total: 17min 7s
Wall time: 17min 19s


In [ ]:
%%time
run_evals(eval_questions, tru_recorder, auto_merging_engine_1)

In [38]:
from trulens_eval import Tru
Tru().get_leaderboard(app_ids=[])

,Context Relevance,Groundedness,Answer Relevance,latency,total_cost
app_id,,,,,
auto merging engine 1,0.171667,0.908333,0.99,30.5,0.001289
auto mergine engine 0,0.125000,0.937500,1.00,30.6,0.005005


In [39]:
Tru().run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   Submit this IP Address: 34.91.164.32



<Popen: returncode: 0 args: ['streamlit', 'run', '--server.headless=True', '...>

In [40]:
## display output texts
import textwrap
records, feedback = Tru().get_records_and_feedback(app_ids=[]) ## "records" is a pandas dataframe
n_rows = 10
for i in range(n_rows):
    input, output = records.loc[i, ['input', 'output']]
    print('👉', textwrap.fill(input, 100))
    print('▪️', textwrap.fill(output, 100))
    output = records.loc[i+n_rows, 'output']
    print('▪️', textwrap.fill(output, 100), '\n')
## generated with the full book RAG

👉 "What is the purpose of life? (Alma 34)"
▪️ "The purpose of life, according to the given context, is to prepare to meet God and to work out
one's salvation with fear before Him. This is a time for individuals to have faith, repent, and
plant the word of God in their hearts. By doing so, they can experience the joy of God's Son and
have their burdens made light."
▪️ "The purpose of life, according to the given context, is to prepare to meet God and to work out
one's salvation with fear before God. This life is seen as a time for individuals to prepare
themselves through faith and repentance. The eternal plan of redemption is based on these
principles, and it is through the atonement of Jesus Christ that individuals can find joy and have
their burdens made light." 

👉 "Does God know me? (Alma 5:38, 58)"
▪️ "Yes, God knows you."
▪️ "Yes, God knows you." 

👉 "How does God answer prayers? (Enos 1)"
▪️ "God answers prayers by speaking to individuals through their minds. In the case of Enos

## **single questions**  

In [42]:
%%time
question = 'Summarize 1 Nephi Chapter 3.' ## check page 27
response = auto_merging_engine_1.query(question)
display_response(response)

**`Final Response:`** Nephi is making an account of his proceedings in his days.

CPU times: user 8.6 s, sys: 30.1 ms, total: 8.63 s
Wall time: 10.6 s


In [43]:
%%time
question = 'What is the afterlife like according to Alma 40?'
response = auto_merging_engine_1.query(question)
display_response(response)

**`Final Response:`** According to Alma 40, there is a time appointed for men to rise from the dead, and there is a space between the time of death and the resurrection. The afterlife is not explicitly described in this context.

CPU times: user 6.99 s, sys: 24.4 ms, total: 7.02 s
Wall time: 8.83 s


# **others**  

* [javascript 1](https://stackoverflow.com/questions/71456390/how-to-keep-the-google-colab-running-without-disconnecting-in-2022), [javascript 2](https://stackoverflow.com/questions/76595272/how-to-stop-google-colab-from-automatically-disconnecting-from-google-drive) that keeps colab running  

> Keep your session active: Even though this should not be necessary when a script is actively running, you could use a JavaScript code snippet that will press the Colab connect button for you every few minutes. It's not an ideal solution but has been reported to help some users. Please remember this might be against the terms of service, use with caution.  

>You should open the browser's developer tools (usually F12), go to the Console tab, paste the above script and hit Enter.  

```
function ConnectButton(){
    console.log("Connect pushed");
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click()
}
setInterval(ConnectButton,60000);
```

# **\<BOTTOM\>**  